# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value

print("Answer question 1:")
print(sum)

limit = 13
generator_data = list(square_root_generator(limit))

print("Answer question 2:")
print(generator_data[12])

Answer question 1:
8.382332347441762
Answer question 2:
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

sum_age = 0
for person in people_1():
    sum_age += person["Age"]

print("Sum ages people_1:")
print(sum_age)

for person in people_2():
  sum_age += person["Age"]

print("Sum ages people_1 and people_2:")
print(sum_age)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Sum ages people_1:
140
Sum ages people_1 and people_2:
353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [13]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [64]:
import dlt
# to do: homework :)
# def people_1():
#     for i in range(1, 6):
#         yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

pipeline = dlt.pipeline(pipeline_name="people_pipeline",
						destination='duckdb',
						dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(list(people_1()),
                    table_name="people_table",
                    write_disposition="replace",
                    primary_key="id")

info = pipeline.run(list(people_2()),
					table_name="people_table",
					write_disposition="merge",
          primary_key="id")

# show the outcome
# print(info)

with pipeline.sql_client() as client:
    conn = client.native_connection
    rel = conn.sql('SELECT * FROM people_table');
    rel.show()
    rel = conn.sql('SELECT SUM(age) FROM people_table');
    rel.show()

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707613154.5973265 │ iQW9PPHspJF7xA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707613154.5973265 │ KVwrCUTTNAZN5A │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1707613155.4025915 │ PBKfIW8qB+lNRw │ Job_8      │
│     7 │ Person_7 │    37 │ City_B  │ 1707613155.4025915 │ ZJzNbJd3rbH17g │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1707613155.4025915 │ /VTPDDD/YAWoGg │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707613155.4025915 │ cL9xy7u7rCWDJw │ Job_6      │
│     5 │ Person_5 │    35 │ City_B  │ 1707613155.4025915 │ sxDVCg8lHfjIeg │ Job_5      │
│     4 │ 

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX